In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.feature_extraction import *
from sklearn.feature_selection import *
from sklearn.decomposition import *
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import *
from sklearn.naive_bayes import *
# from catboost import CatBoost,CatBoostRegressor
# from xgboost import XGBRegressor,XGBRFRegressor

In [2]:
data = pd.read_csv('./data/train.csv')

In [3]:
test_data = pd.read_csv('./data/test.csv')

In [4]:
data['PassengerId'] = data['PassengerId'].astype(int)

In [5]:
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [6]:
data['RoomService'] = data['RoomService'].fillna(data['RoomService'].mean())
data['FoodCourt'] = data['FoodCourt'].fillna(data['FoodCourt'].mean())
data['ShoppingMall'] = data['ShoppingMall'].fillna(data['ShoppingMall'].mean())
data['Spa'] = data['Spa'].fillna(data['Spa'].mean())
data['VRDeck'] = data['VRDeck'].fillna(data['VRDeck'].mean())

In [7]:
data.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [8]:
new_transported = []
for transport in data['Transported']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)

In [9]:
data['Transported'] = new_transported

In [10]:
new_transported = []
for transport in data['CryoSleep']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)

In [11]:
data['CryoSleep'] = new_transported

In [12]:
new_transported = []
for transport in data['VIP']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)

In [13]:
data['VIP'] = new_transported

In [14]:
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,101,Europa,1.0,B/0/P,TRAPPIST-1e,39.0,1.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,1
1,201,Earth,1.0,F/0/S,TRAPPIST-1e,24.0,1.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,0
2,301,Europa,1.0,A/0/S,TRAPPIST-1e,58.0,0.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,1
3,302,Europa,1.0,A/0/S,TRAPPIST-1e,33.0,1.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,1
4,401,Earth,1.0,F/1/S,TRAPPIST-1e,16.0,1.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,927601,Europa,1.0,A/98/P,55 Cancri e,41.0,0.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,1
8689,927801,Earth,0.0,G/1499/S,PSO J318.5-22,18.0,1.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,1
8690,927901,Earth,1.0,G/1500/S,TRAPPIST-1e,26.0,1.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,0
8691,928001,Europa,1.0,E/608/S,55 Cancri e,32.0,1.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,1


In [15]:
data = data.drop(['PassengerId','VIP','FoodCourt','ShoppingMall'],axis=1)

In [16]:
idx = 0
labels = {}
for label in data['HomePlanet']:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx

In [17]:
labels

{'Europa': 1, 'Earth': 2, 'Mars': 3, nan: 4}

In [18]:
new_data = []
for i in data['HomePlanet']:
    new_data.append(labels[i])

In [19]:
data['HomePlanet'] = new_data

In [20]:
new_cabins = []
for c in data['Cabin']:
    if type(c) is float:
        new_cabins.append(None)
    else:
        c = c.split('/')[1]
        new_cabins.append(int(c))

In [21]:
data['Cabin'] = new_cabins

In [22]:
data['Cabin'] = data['Cabin'].fillna(data['Cabin'].mean())

In [23]:
data.corr()

,HomePlanet,CryoSleep,Cabin,Age,RoomService,Spa,VRDeck,Transported
HomePlanet,1.000000,0.035693,0.323321,-0.118649,0.171670,-0.208472,-0.227421,0.091774
CryoSleep,0.035693,1.000000,0.037089,0.073396,0.249645,0.201819,0.196764,0.468645
Cabin,0.323321,0.037089,1.000000,-0.129538,-0.008454,-0.126451,-0.136526,0.044578
Age,-0.118649,0.073396,-0.129538,1.000000,0.067612,0.120992,0.099210,0.074249
RoomService,0.171670,0.249645,-0.008454,0.067612,1.000000,0.009925,-0.019207,0.242048
Spa,-0.208472,0.201819,-0.126451,0.120992,0.009925,1.000000,0.147957,0.218791
VRDeck,-0.227421,0.196764,-0.136526,0.099210,-0.019207,0.147957,1.000000,0.204825
Transported,0.091774,0.468645,0.044578,0.074249,0.242048,0.218791,0.204825,1.000000


In [24]:
data['CryoSleep'] = data['CryoSleep'].fillna(data['CryoSleep'].mean())

In [25]:
data = data.drop(['Destination','Name'],axis=1)

In [26]:
data

,HomePlanet,CryoSleep,Cabin,Age,RoomService,Spa,VRDeck,Transported
0,1,1.0,0.0,39.0,0.0,0.0,0.0,1
1,2,1.0,0.0,24.0,109.0,549.0,44.0,0
2,1,1.0,0.0,58.0,43.0,6715.0,49.0,1
3,1,1.0,0.0,33.0,0.0,3329.0,193.0,1
4,2,1.0,1.0,16.0,303.0,565.0,2.0,0
...,...,...,...,...,...,...,...,...
8688,1,1.0,98.0,41.0,0.0,1643.0,74.0,1
8689,2,0.0,1499.0,18.0,0.0,0.0,0.0,1
8690,2,1.0,1500.0,26.0,0.0,1.0,0.0,0
8691,1,1.0,608.0,32.0,0.0,353.0,3235.0,1


In [27]:
X = data.drop('Transported',axis=1)
y = data['Transported']

In [28]:
model = RandomForestClassifier()
model.fit(X,y)

RandomForestClassifier()

In [29]:
X

,HomePlanet,CryoSleep,Cabin,Age,RoomService,Spa,VRDeck
0,1,1.0,0.0,39.0,0.0,0.0,0.0
1,2,1.0,0.0,24.0,109.0,549.0,44.0
2,1,1.0,0.0,58.0,43.0,6715.0,49.0
3,1,1.0,0.0,33.0,0.0,3329.0,193.0
4,2,1.0,1.0,16.0,303.0,565.0,2.0
...,...,...,...,...,...,...,...
8688,1,1.0,98.0,41.0,0.0,1643.0,74.0
8689,2,0.0,1499.0,18.0,0.0,0.0,0.0
8690,2,1.0,1500.0,26.0,0.0,1.0,0.0
8691,1,1.0,608.0,32.0,0.0,353.0,3235.0


In [30]:
preds = model.predict(np.array(X))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [31]:
np.mean(cross_val_score(model,np.array(X),np.array(y),cv=25))

0.7462479711153068